In [1]:
import sys
sys.path.insert(1, "../")

import importlib
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import numpy as np
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
def reload_modules():
    import ghost_unfairness
    importlib.reload(ghost_unfairness.fair_dataset)
    importlib.reload(ghost_unfairness.utils)
    
reload_modules()

from ghost_unfairness.fair_dataset import FairDataset, default_mappings

from ghost_unfairness.utils import *

In [3]:
protected = ["sex"]
privileged_classes = [['Male']]
metadata = default_mappings.copy()
metadata['protected_attribute_maps'] = [{1.0: 'Male', 0.0: 'Female'}]

alpha = 0.5
beta = 1

kwargs = {'protected_attribute_names': protected,
          'privileged_classes': [['Male']],
          'metadata':metadata,
          'favorable_classes': [1],
          'beta': beta,
          'alpha': alpha
         }

random_state = 47

In [4]:
model_type = GaussianNB

In [5]:
train_fd, test_fd = get_datasets(10000, 1, kwargs, test_random_state=23)
pmod, pmod_results = get_groupwise_preformance(train_fd, test_fd, model_type,
                                               privileged=True)
umod, umod_results = get_groupwise_preformance(train_fd, test_fd, model_type,
                                               privileged=False)
mod, mod_results = get_groupwise_preformance(train_fd, test_fd, model_type,
                                             privileged=None)

In [6]:
print('pmod_results', *pmod_results, sep='\t')
print('umod_results', *umod_results, sep='\t')
print('mod_results', *mod_results, sep='\t')

pmod_results	-0.14939999999999998	0.7006012024048097	0.8955	0.995	0.796
umod_results	-0.07740000000000002	0.8665977249224405	0.8793	0.9198	0.8388
mod_results	-0.09419999999999995	0.8183571153104513	0.9037	0.981	0.8264


In [7]:
df, _ = train_fd.convert_to_dataframe()
for s, group in df.groupby(['sex']):
    print(group.describe())

                  0      sex         label
count  10000.000000  10000.0  10000.000000
mean       4.961304      0.0      0.500000
std        7.097277      0.0      0.500025
min      -18.399566      0.0      0.000000
25%       -0.290851      0.0      0.000000
50%        5.104661      0.0      0.500000
75%       10.160836      0.0      1.000000
max       31.222150      0.0      1.000000
                  0      sex         label
count  10000.000000  10000.0  10000.000000
mean       8.005930      1.0      0.500000
std        5.374212      0.0      0.500025
min       -3.772758      1.0      0.000000
25%        3.025029      1.0      0.000000
50%        8.007648      1.0      0.500000
75%       12.999936      1.0      1.000000
max       19.927426      1.0      1.000000


In [8]:
from aif360.algorithms.preprocessing import DisparateImpactRemover

di = DisparateImpactRemover(repair_level=1)
train_repd = di.fit_transform(train_fd)
test_repd = di.fit_transform(test_fd)
model_type = GaussianNB
rep_pmod, rep_pmod_results = get_groupwise_preformance(train_repd, test_repd, model_type,
                                               privileged=True)
rep_umod, rep_umod_results = get_groupwise_preformance(train_repd, test_repd, model_type,
                                               privileged=False)
rep_mod, rep_mod_results = get_groupwise_preformance(train_repd, test_repd, model_type,
                                             privileged=None)

In [9]:
print('pmod_results', *rep_pmod_results, sep='\t')
print('umod_results', *rep_umod_results, sep='\t')
print('mod_results', *rep_mod_results, sep='\t')

pmod_results	0.0	1.0	0.9156	0.9918	0.8394
umod_results	0.0	1.0	0.9152	0.9912	0.8392
mod_results	0.0	1.0	0.9168	0.9948	0.8388


In [10]:
df, _ = train_repd.convert_to_dataframe()
for s, group in df.groupby(['sex']):
    print(group.describe())

                  0      sex         label
count  10000.000000  10000.0  10000.000000
mean       4.830502      0.0      0.500000
std        6.848710      0.0      0.500025
min      -18.399566      0.0      0.000000
25%       -0.290851      0.0      0.000000
50%        5.104661      0.0      0.500000
75%       10.160836      0.0      1.000000
max       19.927426      0.0      1.000000
                  0      sex         label
count  10000.000000  10000.0  10000.000000
mean       4.830502      1.0      0.500000
std        6.848710      0.0      0.500025
min      -18.399566      1.0      0.000000
25%       -0.290851      1.0      0.000000
50%        5.104661      1.0      0.500000
75%       10.160836      1.0      1.000000
max       19.927426      1.0      1.000000


In [11]:
print(np.sqrt(pmod.sigma_))
print(np.sqrt(umod.sigma_))
print(np.sqrt(mod.sigma_))

print()
print(pmod.theta_)
print(umod.theta_)
print(mod.theta_)

[[1.99842399]
 [1.99701377]]
[[5.11499565]
 [4.97410084]]
[[4.17130264]
 [4.0839225 ]]

[[ 3.01710348]
 [12.99475632]]
[[-0.03007637]
 [ 9.95268372]]
[[ 1.49351355]
 [11.47372002]]


In [12]:
print(np.sqrt(rep_pmod.sigma_))
print(np.sqrt(rep_umod.sigma_))
print(np.sqrt(rep_mod.sigma_))

print()
print(rep_pmod.theta_)
print(rep_umod.theta_)
print(rep_mod.theta_)

[[4.08745357]
 [3.4708501 ]]
[[5.11340262]
 [4.51584763]]
[[4.64803627]
 [4.04932089]]

[[-0.87240052]
 [10.53340505]]
[[-0.03059387]
 [ 9.6915984 ]]
[[-0.4514972 ]
 [10.11250172]]


In [13]:
import scipy.stats as stats
import math

def plot_normal(mu, sigma, label=None):
    x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
    plt.plot(x, stats.norm.pdf(x, mu, sigma), label=label)
    
def plot_non_linear_boundary(mu1, mu2, sigma1, sigma2, p, d, label=None):
    x = np.linspace(-200, 200, 10000)
    y = np.log(p/(1-p)) - d*np.log(sigma1/sigma1) 
    y -= 1/(2*sigma1**2)*(x-mu1)**2 
    y += 1/(2*sigma2**2)*(x-mu2)**2
    plt.plot(x, y, label=label)
    plt.legend()

In [14]:
plot_non_linear_boundary(1.5, 11.5, 4.15, 4.15, 0.5, 1, 'Boundary')

In [15]:
plot_normal(-3, 2, label='P Neg')
plot_normal(13, 2, label='P Pos')
plt.vlines(5, 0, 0.2, label = 'P Boundary')
plt.xlim(-15, 25)
plt.ylim(-0.005, 0.205)
plt.legend()
plt.tight_layout()

In [16]:
plot_normal(0, 5, label='P Neg')
plot_normal(10, 5, label='P Pos')
plt.vlines(5, 0, 0.2, label = 'P Boundary')
plt.xlim(-15, 25)
plt.ylim(-0.005, 0.205)
plt.legend()
plt.tight_layout()

In [17]:
plot_normal(1.5, 4.09, label='Pos')
plot_normal(11.5, 4.09, label='Neg')
plt.vlines(6.5, 0, 0.2, label='Boundary')
plt.xlim(-15, 25)
plt.ylim(-0.005, 0.205)
plt.legend()
plt.tight_layout()

In [18]:
plot_normal(3, 2)
plot_normal(13, 2)
# plt.vlines(6.5, 0, 0.2)
# plot_normal(0, 5)
# plot_normal(10, 5)
plt.vlines(5.5, 0, 0.2)
plt.vlines(6.5, 0, 0.2)
plt.xlim(0, 15)
plt.ylim(0, 0.2)

(0.0, 0.2)

In [19]:
# plot_normal(3, 2)
# plot_normal(10, 2)
# plt.vlines(6.5, 0, 0.2)
plot_normal(0, 5)
plot_normal(10, 5)
plt.vlines(5, 0, 0.2)
plt.vlines(6.5, 0, 0.2)
plt.xlim(0, 15)
plt.ylim(0, 0.2)

(0.0, 0.2)

In [20]:
model_type = LogisticRegression
train_fd, test_fd = get_datasets(10000, 2, kwargs, test_random_state=23)
pmod, pmod_results = get_groupwise_preformance(train_fd, test_fd, model_type,
                                               privileged=True)
umod, umod_results = get_groupwise_preformance(train_fd, test_fd, model_type,
                                               privileged=False)
mod, mod_results = get_groupwise_preformance(train_fd, test_fd, model_type,
                                             privileged=None)

In [21]:
plot_lr_boundary(pmod, plt, "pos")
plot_lr_boundary(umod, plt, "neg")
plot_lr_boundary(mod, plt, "mod")
plt.show()

<ipython-input-21-a512f64947f5>:4: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [22]:
print("Loss D_p t_p", 1 - pmod_results[3])
print("Loss D_u t_u", 1 - umod_results[4])
print("Loss D t", 1 - mod_results[2])

Loss D_p t_p 0.00019999999999997797
Loss D_u t_u 0.07540000000000002
Loss D t 0.04530000000000001


In [23]:
mods = [pmod, umod, mod]
mods_results = [pmod_results, umod_results, mod_results]
theta_p = None
theta_u = None
theta = None
min_loss_p = 1
min_loss_u = 1
min_loss = 1
for i in range(len(mods)):
    if (1 - mods_results[i][3]) < min_loss_p:
        theta_p = mods[i]
        min_loss_p = 1 - mods_results[i][3]
    if (1 - mods_results[i][4]) < min_loss_u:
        theta_u = mods[i]
        min_loss_u = 1 - mods_results[i][4]
        
    if (1 - mods_results[i][2]) < min_loss:
        theta = mods[i]
        min_loss = 1 - mods_results[i][2]
        
print(1-min_loss_p, 1-min_loss_u, 1-min_loss)
print((beta*min_loss_p + 1*min_loss_u)/(1+beta))
print(min_loss)
assert round(min_loss, 8) - round((beta*min_loss_p + 1*min_loss_u)/(1+beta), 8) >= 0

0.9998 0.9246 0.9547
0.0378
0.04530000000000001


In [24]:
ptrain_results = get_classifier_metrics(pmod, train_fd)

print(ptrain_results)
utrain_results = get_classifier_metrics(umod, train_fd)

print(utrain_results)
train_results = get_classifier_metrics(mod, train_fd)

print(train_results)

[-0.12519999999999998, 0.7496, 0.9315, 0.9998, 0.8632]
[-0.03469999999999995, 0.9352128454070202, 0.94535, 0.9644, 0.9263]
[-0.04629999999999995, 0.909126594700687, 0.95315, 0.9905, 0.9158]


In [25]:
plot_lr_boundary(pmod, plt, '_'.join([str(x) for x in [alpha, beta, 0.88]]))
plot_lr_boundary(umod, plt, '_'.join([str(x) for x in [alpha, beta, 0.94]]))
plot_lr_boundary(mod, plt, '_'.join([str(x) for x in [alpha, beta, 0.92]]))
test_fd_x, test_fd_y = train_fd.get_xy(keep_protected=False)
plt.scatter(test_fd_x['0'], test_fd_x['1'], c=test_fd_y, label=None)
plt.xlim(-10, 20)
plt.ylim(-10, 20)
plt.legend()

In [26]:
dict(zip(test_fd.protected_attribute_names, test_fd.privileged_protected_attributes))

{'sex': array([1.])}

In [27]:
import copy

def get_edge_lr(model, end, interval, criteria):
    cms = {}
    c = 0
    while True:
        temp_mod = copy.deepcopy(model)
        temp_mod.intercept_ = [i + c for i in temp_mod.intercept_]
        cm = get_classifier_metrics(temp_mod, test_fd)

        cms[c] = cm
        if cm[criteria] < end:
            break
        c -= interval
        
    return temp_mod, cms

    
def get_lr_boundary_range(model, criteria):
    cms = {}
    temp_mods = [copy.deepcopy(model)]
    left_mod, cms_left = get_edge_lr(model, 0.6, -0.1, criteria)
    temp_mods.append(left_mod)
    cms = cms_left.copy()
    right_mod, cms_right = get_edge_lr(model, 0.6, 0.1, criteria)
    temp_mods.append(right_mod)
    cms.update(cms_right)

    for i, lr in enumerate(temp_mods):
        plot_lr_boundary(lr, plt, str(i))
        
    return temp_mods, cms

In [28]:
def plot_cms(cms):
    skeys = sorted(cms.keys())
    accs = [cms[i][3] for i in skeys]
    plt.plot(skeys, accs, label='Pos Acc')

    accs = [cms[i][4] for i in skeys]
    plt.plot(skeys, accs, label='Neg Acc')
    plt.legend()

    accs = [cms[i][2] for i in skeys]
    plt.plot(skeys, accs, label='Ov Acc')
    plt.legend()

In [29]:
temp_mods, cms = get_lr_boundary_range(pmod, 3)
plt.show()
plot_cms(cms)

<ipython-input-29-3119d761def3>:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [30]:
temp_mods, cms = get_lr_boundary_range(umod, 4)
plt.show()
plot_cms(cms)

<ipython-input-30-9d5aa5e684aa>:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [31]:
temp_mods, cms = get_lr_boundary_range(mod, 2)
plt.show()
plot_cms(cms)

<ipython-input-31-1f2928bf5776>:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [32]:
gammas = [0, 0.2, 0.4, 0.6, 0.8, 1]

cms = {}

for g in gammas:
    temp_mod = copy.deepcopy(pmod)
    intercepts = np.add((1-g)*pmod.intercept_, g*umod.intercept_)
    temp_mod.intercept_ = intercepts
    coefs = np.add((1-g)*pmod.coef_, g*umod.coef_)
    temp_mod.coef_ = coefs
    cm = get_classifier_metrics(temp_mod, test_fd)
    cms[g] = cm
    plot_lr_boundary(temp_mod, plt, None)
    
    
plt.xlim(-10, -5)
plt.ylim(20, 25)
plt.show()
    
skeys = sorted(cms.keys())
accs = [cms[i][3] for i in skeys]
plt.plot(skeys, accs, '*-', label='Pos Acc')

accs = [cms[i][4] for i in skeys]
plt.plot(skeys, accs, '*-', label='Neg Acc')
plt.legend()

accs = [cms[i][2] for i in skeys]
plt.plot(skeys, accs, '*-', label='Ov Acc')
plt.legend()

# plt.hlines(0.935, 0, 1)

<ipython-input-32-aa646b921498>:18: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [33]:
print(*[cms[i] for i in cms], sep='\n')

[-0.12079999999999996, 0.7584966013594563, 0.9326, 0.9998, 0.8654]
[-0.10439999999999999, 0.7912834866053579, 0.9388, 0.9998, 0.8778]
[-0.08580000000000004, 0.8286056731921694, 0.9471, 0.9994, 0.8948]
[-0.06799999999999995, 0.8646496815286625, 0.9536, 0.9976, 0.9096]
[-0.050399999999999945, 0.9010600706713782, 0.9548, 0.9906, 0.919]
[-0.03760000000000002, 0.9297196261682242, 0.9448, 0.965, 0.9246]


In [34]:
p = [cms[i][3] for i in cms]
u = [cms[i][4] for i in cms]
u.reverse()
print(p)
print(u)
print([p[0] - p[i] for i in range(1, len(p))])
print([u[0] - u[i] for i in range(1, len(u))])

[0.9998, 0.9998, 0.9994, 0.9976, 0.9906, 0.965]
[0.9246, 0.919, 0.9096, 0.8948, 0.8778, 0.8654]
[0.0, 0.00040000000000006697, 0.0021999999999999797, 0.009199999999999986, 0.03480000000000005]
[0.005599999999999938, 0.015000000000000013, 0.029799999999999938, 0.04679999999999995, 0.05920000000000003]


1. First get the decision boundary (Use NB or LR. maybe both are doable.)
2. Using the boundary get $P(\hat{y} = 1| u/p, \theta_u/\theta_p)$.
3. Show that, $DI(\theta_p)$ < $DI(\theta)$ < $DI(\theta_u)$

In [35]:
from scipy.stats import norm
def get_positive_prediction_rate(mu1, mu2, sigma1, sigma2, mu_avg):
    rv1 = norm(loc=mu1, scale=sigma1)
    # print(rv1.cdf(mu_avg))
    rv2 = norm(loc=mu2, scale=sigma2)
    # print(rv2.cdf(mu_avg))

    return (1 - 0.5*rv1.cdf(mu_avg) - 0.5 * rv2.cdf(mu_avg))

In [36]:
print(get_positive_prediction_rate(0, 9, 5, 5, 6.5))
print(get_positive_prediction_rate(3, 10, 2, 2, 6.5))

0.3941314729298117
0.5


In [37]:
0.3941314729298117/0.5

0.7882629458596234

In [38]:
print(get_positive_prediction_rate(0, 9, 5, 5, 5))
print(get_positive_prediction_rate(3, 10, 2, 2, 5))
get_positive_prediction_rate(0, 9, 5, 5, 5)/get_positive_prediction_rate(3, 10, 2, 2, 5)

0.4733999276740303
0.5762227943028405


0.8215570997096472

In [39]:
0.5/0.577

0.8665511265164645

In [40]:
print(get_positive_prediction_rate(0, 10, 5, 5, 5.495))
print(get_positive_prediction_rate(3, 10, 2, 2, 5.495))
get_positive_prediction_rate(0, 10, 5, 5, 5.495)/get_positive_prediction_rate(3, 10, 2, 2, 5.495)

0.4760449365530461
0.5469808818805002


0.8703136660214176

In [41]:
get_positive_prediction_rate(0, 10, 5, 5, 5.45916)/get_positive_prediction_rate(3, 10, 2, 2, 5.45916)

0.8704019599082464

In [42]:
print(get_positive_prediction_rate(0, 9, 5, 5, 5.75))
print(get_positive_prediction_rate(3, 10, 2, 2, 5.75))
get_positive_prediction_rate(0, 9, 5, 5, 5.75)/get_positive_prediction_rate(3, 10, 2, 2, 5.75)

0.4336129124156428
0.5338862079514434


0.8121822702246686

In [43]:
0.4637/0.5333

0.8694918432402026

In [44]:
i = 5
ratios = []
while i <= 8:
    ratio = get_positive_prediction_rate(0, 10, 5, 5, i)/get_positive_prediction_rate(3, 13, 2, 2, i)
    ratios.append(ratio)
    print(i, ratio, sep='\t\t')
    i+= 0.25
    
plt.plot(np.arange(5, 8.25, 0.25), ratios)

5		0.8630930829826579
5.25		0.8633581273930458
5.5		0.8607445711688895
5.75		0.8552113854984534
6.0		0.8468364943145887
6.25		0.8358012966730142
6.5		0.8223679909385787
6.75		0.8068537748189683
7.0		0.7896057928155817
7.25		0.7709797585841528
7.5		0.7513238914857889
7.75		0.7309685558482791
8.0		0.7102210333098821


In [45]:
i = 5
ratios = []
while i <= 6.5:
    ratio = get_positive_prediction_rate(0, 9, 5, 5, i)/get_positive_prediction_rate(3, 10, 2, 2, i)
    ratios.append(ratio)
    print(i, ratio, sep='\t\t')
    i+= 0.25
    
plt.plot(np.arange(5, 6.75,0.25), ratios)

5		0.8215570997096472
5.25		0.8205218538264236
5.5		0.8173419875105453
5.75		0.8121822702246686
6.0		0.8053358165743911
6.25		0.7972028096019748
6.5		0.7882629458596234


In [46]:
get_positive_prediction_rate(0, 9, 5, 5, 5.495)/get_positive_prediction_rate(3, 10, 2, 2, 5.495)

0.8174257592607983

In [47]:
print(get_positive_prediction_rate(0, 10, 5, 5, 8))
print(get_positive_prediction_rate(3, 13, 2, 2, 8))
get_positive_prediction_rate(0, 10, 5, 5, 8)/get_positive_prediction_rate(3, 13, 2, 2, 8)

0.35511051665494103
0.5


0.7102210333098821

In [48]:
print(get_positive_prediction_rate(0, 10, 5, 5, 5))
print(get_positive_prediction_rate(3, 13, 2, 2, 5))
get_positive_prediction_rate(0, 10, 5, 5, 5)/get_positive_prediction_rate(3, 13, 2, 2, 5)

0.5
0.579311791344812


0.8630930829826579

In [49]:
print(get_positive_prediction_rate(0, 10, 5, 5, 6.5))
print(get_positive_prediction_rate(3, 13, 2, 2, 6.5))
get_positive_prediction_rate(0, 10, 5, 5, 6.5)/get_positive_prediction_rate(3, 13, 2, 2, 6.5)

0.42741841618126863
0.5197410659107132


0.8223679909385787